# Evaluation LDA

- kombi of different class und topic
- mapping averbis klasse mit meiner Zahl
- Fehlerrrberechnung - erhalte Zahl ob korrekt oder nicht

In [1]:
# imports
import pandas as pd
import numpy as np
import itertools
from itertools import permutations
from sklearn import metrics

In [2]:
#read csv file from predicted topics
df = pd.read_csv('/home/ubuntu/ullrich/my_code/data/predicted_LDA.csv', sep=',')

In [3]:
# drop unnessecary columns 
df = df.drop(['Unnamed: 0'], axis=1)
# replace topic name
df = df.replace({'class':'ErnÃ¤hrung'},{'class':'Ernährung'})

In [4]:
df

,dbrecordid,combined,topic,certainty,class
0,AGRICOLAIND20439684,"genetic markers, products, glycine max, progen...",4.0,0.524350,Landwirtschaft
1,M35399217,"evaluation, momordica charantia, methods, glut...",0.0,0.985263,Ernährung
2,AGRICOLAIND605620910,"economic distribution, approximation, approxim...",3.0,0.754991,Rest
3,M7452196,"environment, experimentation, iris (eye), iris...",2.0,0.981984,Umweltwissenschaften
4,M7263312,"ghana, burkitt lymphoma, burkitt lymphoma, pat...",3.0,0.962820,Rest
...,...,...,...,...,...
119630,BASE::ftdoajarticles:oai:doaj.org/article:dc57...,"equipment, drugs, redmouth disease, statistica...",NaN,NaN,Rest
119631,BASE::ftdoajarticles:oai:doaj.org/article:90c1...,"buildings, ghana, housing, buildings, building...",0.0,0.807834,Umweltwissenschaften
119632,M3100915,"shigella dysenteriae, shiga toxin, shiga toxin...",NaN,NaN,Ernährung
119633,TIB729405060,"globalization, globalization, globalization, i...",1.0,0.559160,Umweltwissenschaften


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119635 entries, 0 to 119634
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   dbrecordid  119635 non-null  object 
 1   combined    119635 non-null  object 
 2   topic       77635 non-null   float64
 3   certainty   77635 non-null   float64
 4   class       119635 non-null  object 
dtypes: float64(2), object(3)
memory usage: 4.6+ MB


- 42.000 none values ?? - Text beinhaltet kein Wort aus der Themenklasse - Beschiss
- unter 60 % raus 

In [ ]:
df_test = df[(df['certainty'] is not None) & (df['certainty'] >= 0.6 )]

In [6]:
#define all possible combinations of the classes
classes = ['Medizin', 'Landwirtschaft', 'Umweltwissenschaften', 'Ernährung', 'Rest']
topics = [0.0,1.0,2.0,3.0,4.0]

unique_combinations = []
permut = itertools.permutations(classes, len(topics))

for comb in permut:
    zipped = zip(comb, topics)
    unique_combinations.append(list(zipped))
 
# printing unique_combination list
print(unique_combinations)

[[('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Umweltwissenschaften', 2.0), ('Ernährung', 3.0), ('Rest', 4.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Umweltwissenschaften', 2.0), ('Rest', 3.0), ('Ernährung', 4.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Ernährung', 2.0), ('Umweltwissenschaften', 3.0), ('Rest', 4.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Ernährung', 2.0), ('Rest', 3.0), ('Umweltwissenschaften', 4.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Rest', 2.0), ('Umweltwissenschaften', 3.0), ('Ernährung', 4.0)], [('Medizin', 0.0), ('Landwirtschaft', 1.0), ('Rest', 2.0), ('Ernährung', 3.0), ('Umweltwissenschaften', 4.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Landwirtschaft', 2.0), ('Ernährung', 3.0), ('Rest', 4.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Landwirtschaft', 2.0), ('Rest', 3.0), ('Ernährung', 4.0)], [('Medizin', 0.0), ('Umweltwissenschaften', 1.0), ('Ernährung', 2.0), ('Landwirtschaft', 3.0), ('Rest', 4.0)], 

In [8]:
#get lists of pred and true values
pred_test = df['topic'].values.tolist()
true_test = df['class'].values.tolist()

In [9]:
#replace items in predicted list
def replace_items(pred_list, true_list):
    liste = list(pred_list)
    for i in range(len(liste)):
        for tupel in true_list:
            if liste[i] == tupel[1]:
                liste[i] = tupel[0]
    return liste

In [10]:
#create f1-score for every combination
def f1_score(combinations, pred_list, true_list):
    result = []
    for combi in combinations:
        res = {}
        pred = replace_items(pred_list,combi)
        f1 = metrics.f1_score(true_list, pred, average='weighted')
        res.update({'Combi': combi,'f1_score': f1})
        result.append(res)
    return result

In [11]:
#get dictionary for each combination and every score
res_dict = f1_score(unique_combinations, pred_test, true_test)

In [ ]:
[x['f1_score'] for x in res_dict]

In [12]:
#get highest value of dictionary
def highest_val(lst, key, key2):
    highest_value = None
    for dict in lst:
        score = dict[key]
        combi = dict[key2]  
        if highest_value is None or score > highest_value:
            highest_value = score
            pred_combi = combi
    return highest_value, pred_combi

In [13]:
f1 = highest_val(res_dict,'f1_score','Combi')

In [14]:
f1

(0.18387745513038994,
 [('Ernährung', 0.0),
  ('Medizin', 1.0),
  ('Rest', 2.0),
  ('Landwirtschaft', 3.0),
  ('Umweltwissenschaften', 4.0)])

In [ ]:
#get list of the class and predicted topic
pred_averbis = df[['class', 'topic']].values.tolist()
pred_averbis_test = df_test[['class', 'topic']].values.tolist()

In [ ]:
def calculate_acc_score(pred_class, combinations):
    result = [] 
    for combi in combinations:
        tmp = {}
        total_count = len(pred_class)
        match_count = 0
        #print(combi)
        for item in pred_class:
            item = tuple(item)
            if item in combi:
                match_count = match_count + 1
        #print(match_count)        
        accuracy = match_count / total_count
        tmp.update({'Combi': combi,'match_count': match_count, 'acc_score': accuracy})
        result.append(tmp)
    return result

In [ ]:
#get dict of results
result = calculate_acc_score(pred_averbis, unique_combinations)
result_test = calculate_acc_score(pred_averbis_test, unique_combinations)

In [ ]:
highest_val(result, 'acc_score', 'Combi')
highest_val(result_test, 'acc_score', 'Combi')